# SOLAQUA Video Export Demo

This notebook demonstrates how to export **camera topics** from `*_video.bag` files to **MP4** or **PNG sequences**, and how to dump **CameraInfo** YAML.

In [1]:
# Ensure project root is on sys.path (edit if needed)
import sys
from pathlib import Path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
print("Working directory:", Path.cwd())


Working directory: /Users/eirikvarnes/code/SOLAQUA


In [2]:
from utils.video_export import (
    list_camera_topics_in_bag,
    export_all_video_bags_to_mp4, export_all_video_bags_to_png,
    export_camera_topic_to_mp4, export_camera_topic_to_png_sequence,
    export_camera_info_for_bag,
)
from utils.core import find_video_bags, sanitize_topic
from pathlib import Path
import pandas as pd


## Configure paths

In [3]:
DATA_DIR = Path("data")  # <-- change to your folder with *_video.bag files
EXPORTS_DIR = Path("exports")
EXPORTS_DIR.mkdir(parents=True, exist_ok=True)

print("Data dir:", DATA_DIR.resolve())
print("Exports dir:", EXPORTS_DIR.resolve())
bags = find_video_bags(DATA_DIR, recursive=True)
bags[:5]


Data dir: /Users/eirikvarnes/code/SOLAQUA/data
Exports dir: /Users/eirikvarnes/code/SOLAQUA/exports


[PosixPath('data/2024-08-20_13-39-34_video.bag'),
 PosixPath('data/2024-08-20_13-40-35_video.bag'),
 PosixPath('data/2024-08-22_14-06-43_video.bag'),
 PosixPath('data/2024-08-22_14-29-05_video.bag'),
 PosixPath('data/2024-08-22_14-47-39_video.bag')]

## List camera topics in one bag

In [4]:
if bags:
    bagpath = bags[0]
    print("Inspecting:", bagpath.name)
    list_camera_topics_in_bag(bagpath)
else:
    print("No *_video.bag files found under", DATA_DIR)


Inspecting: 2024-08-20_13-39-34_video.bag


## Export CameraInfo YAML

In [5]:
if bags:
    df_info = export_camera_info_for_bag(bags[0], out_dir=EXPORTS_DIR / "camera_info")
    display(df_info.head(10))
else:
    print("Skipping CameraInfo export.")


,bag,topic,out_file
0,2024-08-20_13-39-34_video,/image/compressed_image/camera_info,exports/camera_info/2024-08-20_13-39-34_video_...


## Export a single camera topic to MP4

In [6]:
if bags:
    bag = bags[0]
    topics = [t for (t, _mt) in list_camera_topics_in_bag(bag)]
    if topics:
        topic = topics[0]
        mp4_path = EXPORTS_DIR / "videos" / f"{bag.stem}__{sanitize_topic(topic)}.mp4"
        res = export_camera_topic_to_mp4(bag, topic, mp4_path, target_fps=None)  # auto-estimate FPS
        res
    else:
        print("No camera topics found in", bag.name)
else:
    print("No video bags found to export from.")


## Export **all** video bags to MP4

In [7]:
if bags:
    idx_mp4 = export_all_video_bags_to_mp4(DATA_DIR, out_dir=EXPORTS_DIR, recursive=True)
    display(idx_mp4.head(20))
else:
    print("No video bags found.")


(skip) exists: exports/videos/2024-08-20_13-39-34_video__image_compressed_image_data.mp4
Wrote exports/videos/2024-08-20_13-39-34_video__ted_image.mp4 (1121 frames @ ~23.02 FPS).
Wrote exports/videos/2024-08-20_13-40-35_video__image_compressed_image_data.mp4 (1184 frames @ ~22.53 FPS).
Wrote exports/videos/2024-08-20_13-40-35_video__ted_image.mp4 (1187 frames @ ~22.76 FPS).
Wrote exports/videos/2024-08-22_14-06-43_video__image_compressed_image_data.mp4 (945 frames @ ~22.70 FPS).
Wrote exports/videos/2024-08-22_14-29-05_video__image_compressed_image_data.mp4 (1585 frames @ ~22.44 FPS).
Wrote exports/videos/2024-08-22_14-47-39_video__image_compressed_image_data.mp4 (1157 frames @ ~22.54 FPS).

Index written: exports/index_video_mp4.csv (7 rows).


,bag,topic,out_file,frames,fps,codec,skipped
0,2024-08-20_13-39-34_video,/image/compressed_image/data,exports/videos/2024-08-20_13-39-34_video__imag...,NaN,NaN,mp4v,True
1,2024-08-20_13-39-34_video,/ted/image,exports/videos/2024-08-20_13-39-34_video__ted_...,1121.0,23.024879,mp4v,False
2,2024-08-20_13-40-35_video,/image/compressed_image/data,exports/videos/2024-08-20_13-40-35_video__imag...,1184.0,22.533909,mp4v,False
3,2024-08-20_13-40-35_video,/ted/image,exports/videos/2024-08-20_13-40-35_video__ted_...,1187.0,22.763212,mp4v,False
4,2024-08-22_14-06-43_video,/image/compressed_image/data,exports/videos/2024-08-22_14-06-43_video__imag...,945.0,22.703084,mp4v,False
5,2024-08-22_14-29-05_video,/image/compressed_image/data,exports/videos/2024-08-22_14-29-05_video__imag...,1585.0,22.436632,mp4v,False
6,2024-08-22_14-47-39_video,/image/compressed_image/data,exports/videos/2024-08-22_14-47-39_video__imag...,1157.0,22.536088,mp4v,False


## Export PNG frame sequences

In [8]:
if bags:
    idx_png = export_all_video_bags_to_png(
        DATA_DIR, out_dir=EXPORTS_DIR, recursive=True,
        stride=5, limit=None, resize_to=None
    )
    display(idx_png.head(20))
else:
    print("No video bags found.")


Wrote 225 PNGs to exports/frames/2024-08-20_13-39-34_video__image_compressed_image_data with index: exports/frames/2024-08-20_13-39-34_video__image_compressed_image_data/index.csv
Wrote 225 PNGs to exports/frames/2024-08-20_13-39-34_video__image_compressed_image_data.
Wrote 225 PNGs to exports/frames/2024-08-20_13-39-34_video__ted_image with index: exports/frames/2024-08-20_13-39-34_video__ted_image/index.csv
Wrote 225 PNGs to exports/frames/2024-08-20_13-39-34_video__ted_image.
Wrote 237 PNGs to exports/frames/2024-08-20_13-40-35_video__image_compressed_image_data with index: exports/frames/2024-08-20_13-40-35_video__image_compressed_image_data/index.csv
Wrote 237 PNGs to exports/frames/2024-08-20_13-40-35_video__image_compressed_image_data.
Wrote 238 PNGs to exports/frames/2024-08-20_13-40-35_video__ted_image with index: exports/frames/2024-08-20_13-40-35_video__ted_image/index.csv
Wrote 238 PNGs to exports/frames/2024-08-20_13-40-35_video__ted_image.
Wrote 189 PNGs to exports/frames

,bag,topic,out_dir,frames,skipped
0,2024-08-20_13-39-34_video,/image/compressed_image/data,exports/frames/2024-08-20_13-39-34_video__imag...,225,False
1,2024-08-20_13-39-34_video,/ted/image,exports/frames/2024-08-20_13-39-34_video__ted_...,225,False
2,2024-08-20_13-40-35_video,/image/compressed_image/data,exports/frames/2024-08-20_13-40-35_video__imag...,237,False
3,2024-08-20_13-40-35_video,/ted/image,exports/frames/2024-08-20_13-40-35_video__ted_...,238,False
4,2024-08-22_14-06-43_video,/image/compressed_image/data,exports/frames/2024-08-22_14-06-43_video__imag...,189,False
5,2024-08-22_14-29-05_video,/image/compressed_image/data,exports/frames/2024-08-22_14-29-05_video__imag...,317,False
6,2024-08-22_14-47-39_video,/image/compressed_image/data,exports/frames/2024-08-22_14-47-39_video__imag...,232,False
